In [53]:
import geopandas as gpd
import pandas as pd
import numpy as np
import os
import yaml
import math
from shapely import LineString, Point, Polygon, distance

In [54]:
i_line = pd.Index(['id', 'Type', 'Region', 'tags.power', 'tags.voltage', 'tags.name',
       'tags.operator', 'tags.utility', 'tags.cables', 'tags.frequency',
       'tags.source', 'tags.operator:wikidata', 'tags.wires', 'tags.circuits',
       'tags.line', 'other_tags', 'geometry'])

i_subs = pd.Index(['id', 'Type', 'Region', 'tags.location', 'tags.name', 'tags.power',
       'tags.substation', 'tags.voltage', 'tags.operator', 'tags.source',
       'tags.building', 'tags.addr:city', 'tags.operator:wikidata',
       'tags.operator:wikipedia', 'tags.addr:full', 'tags.barrier',
       'other_tags', 'geometry'])

def create_custom_network(df, name):
    df["bus0"] = [Point(xy) for xy in zip(df.x1,df.y1)]
    df["bus1"] = [Point(xy) for xy in zip(df.x2,df.y2)]
    
    # defining custom lines
    df["geometry"] = [LineString(xy) for xy in zip(df['bus0'],df['bus1'])]
    df_line = df[df.columns.intersection(i_line)]
    df_line.reset_index(drop=True)
    df_line["id"] = df_line.index + 100000000
    gdf_line = gpd.GeoDataFrame(df_line, crs="EPSG:4326")
    gdf_line.to_file("custom_lines_" + name + ".geojson", driver="GeoJSON")
    
    # defining custom substations
    df["tags.power"] = "substation"
    df["tags.substation"] = "transmission"
    
    df_sub1 = df.copy(deep=True)
    df_sub1["geometry"] = df_sub1["bus0"]
    
    df_sub2 = df.copy(deep=True)
    df_sub2["Region"] = df_sub2["Region1"]
    df_sub2["geometry"] = df_sub2["bus1"]
    
    df_sub = pd.concat([df_sub1,df_sub2])
    df_sub.reset_index(drop=True)
    df_sub["id"] = df_sub.index + 100000000
    df_sub = df_sub[df_sub.columns.intersection(i_subs)]
    gdf_sub = gpd.GeoDataFrame(df_sub, crs="EPSG:4326")
    gdf_sub.to_file("custom_substations_" + name + ".geojson", driver="GeoJSON")


In [55]:
df_int = pd.read_csv("csv_file/custom_int_net.csv")

for name in ["exist", "aims", "irena"]:
    df_ext = pd.read_csv("csv_file/custom_ext_net_" + name + ".csv")
    df = pd.concat([df_int,df_ext])

    create_custom_network(df, name)

/tmp/ipykernel_4944/2168686921.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_line["id"] = df_line.index + 100000000
/tmp/ipykernel_4944/2168686921.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_line["id"] = df_line.index + 100000000
/tmp/ipykernel_4944/2168686921.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-do